In [52]:
import ftplib
from pathlib import Path
import logging

from collections import defaultdict
logging.basicConfig(level=logging.DEBUG)

In [53]:
FTP_URL = "localhost"
FTP_USER = "ituser"
FTP_PWD = "0000"

dir_remote = "."
DIR_LOCAL = "./down2"


In [ ]:
ftp = ftplib.FTP(FTP_URL, FTP_USER, FTP_PWD, timeout=10)

def recursive_list_dir(ftp: ftplib.FTP, dir: str):
    lst_file = list()
    dic_dir = defaultdict(list)
    cnt_depth = 0

    def _recursive_list_dir(ftp: ftplib.FTP, root: str):
        nonlocal cnt_depth
        ftp.cwd(root)
        for i in ftp.mlsd(facts=["type"]):
            if i[1]["type"] == "dir":
                _dir = i[0]
                print(f"Found Directory : {_dir}")
                dic_dir[cnt_depth].append(_dir)
                cnt_depth+=1
                _recursive_list_dir(ftp, _dir)
                ftp.cwd("..")
            elif i[1]["type"] == "file":
                _file = f"{ftp.pwd()}/{i[0]}"
                print(f"Found File      : {_file}")
                lst_file.append(_file)
            else:
                print(f"Unknown type: {i[0]}")
        cnt_depth-=1
    _recursive_list_dir(ftp, dir)
    return lst_file, dic_dir


lst_file, dic_dir = recursive_list_dir(ftp, dir_remote)
print(lst_file)
print(dic_dir)

# ftp.quit()

Found Directory : test123
Found Directory : l1
Found Directory : empty
Found Directory : l2
Found File      : /test123/l1/l2/fuck.txt
Found File      : /test123/l1/l2_file1.txt
Found File      : /test123/l1/l2_file2.txt
Found File      : /test123/l1_file1.txt
Found File      : /test123/l1_file2.txt
Found File      : //time.txt
['/test123/l1/l2/fuck.txt', '/test123/l1/l2_file1.txt', '/test123/l1/l2_file2.txt', '/test123/l1_file1.txt', '/test123/l1_file2.txt', '//time.txt']
defaultdict(<class 'list'>, {0: ['test123'], 1: ['l1'], 2: ['empty', 'l2']})


In [43]:
ftp = ftplib.FTP(FTP_URL, FTP_USER, FTP_PWD, timeout=10)
for idx, file in enumerate(lst_file):
    if file.startswith("/"):
        path_file = Path(DIR_LOCAL) / file.lstrip("/")
    else:
        path_file = Path(DIR_LOCAL) / file
    print(f"File: {file}\n path_file: {path_file}")
    
    path_file.parent.mkdir(parents=True, exist_ok=True)
    
    try:
        with open(path_file, 'wb') as f:
            print(f"Download: {file} to {path_file}")
            ftp.retrbinary(f"RETR {file}", f.write)
    except Exception as e:
        print(f"Error : {e}")
ftp.quit()

File: /test123/l1/l2/fuck.txt
 path_file: down2\test123\l1\l2\fuck.txt
Download: /test123/l1/l2/fuck.txt to down2\test123\l1\l2\fuck.txt
File: /test123/l1/l2_file1.txt
 path_file: down2\test123\l1\l2_file1.txt
Download: /test123/l1/l2_file1.txt to down2\test123\l1\l2_file1.txt
File: /test123/l1/l2_file2.txt
 path_file: down2\test123\l1\l2_file2.txt
Download: /test123/l1/l2_file2.txt to down2\test123\l1\l2_file2.txt
File: /test123/l1_file1.txt
 path_file: down2\test123\l1_file1.txt
Download: /test123/l1_file1.txt to down2\test123\l1_file1.txt
File: /test123/l1_file2.txt
 path_file: down2\test123\l1_file2.txt
Download: /test123/l1_file2.txt to down2\test123\l1_file2.txt


'221 Service closing control connection.'

In [75]:
# ftp = ftplib.FTP(FTP_URL, FTP_USER, FTP_PWD, timeout=10)
dir_upload = "upload"


def walk_upload_dir(dir_upload):
    path_upload = Path(dir_upload)
    for path in path_upload.glob("*"):
        if path.is_dir():
            print(f"Path {path.as_posix()} is_dir: {path.is_dir()}")
            walk_upload_dir(path)
        else:
            print(f"Upload: {path.as_posix()}")
    
walk_upload_dir(dir_upload)
                  

Path upload/123 is_dir: True
Upload: upload/123/123.txt
Upload: upload/75459241_p0.jpg
Path upload/nothing is_dir: True


In [80]:
ftp = ftplib.FTP(FTP_URL, FTP_USER, FTP_PWD, timeout=10)
ftp.cwd("test123/l1/l2")
print(ftp.pwd())
ftp.cwd("/")
print(ftp.pwd())
ftp.cwd("test123")
print(ftp.pwd())
ftp.quit()

/test123/l1/l2
/
/test123


'221 Service closing control connection.'

'221 Service closing control connection.'